In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json

def import_census_data(vars, target, year, limiter):
  '''
  limiter: geographies/years/certain values to be appended to base request
  '''
  curr = 0
  weight = "PWGTP"
  housing_identifier = "SERIALNO"
  person_identifier = "SPORDER"
  base_request = "https://api.census.gov/data/" + year + "/acs/acs1/pums?get=" + weight + "," + target + "," + housing_identifier + "," + person_identifier
  request = base_request + limiter
  num_vars = 4
  vars.remove(target)
  vars.remove(weight)
  vars.remove(housing_identifier)
  vars.remove(person_identifier)

  #get target/weight
  print(request)
  response = requests.get(request)
  json_data = json.dumps(response.json())
  df = pd.read_json(json_data)
  df = df.rename(columns=df.iloc[0]).loc[1:].reset_index()
  df = df[[target, weight, housing_identifier, person_identifier]]
  print(df) #todo:delete

  target_col = df[target]
  weight_col = df[weight]
  housing_identifier_col = df[housing_identifier]
  person_identifier_col = df[person_identifier]

  #requesting all data #todo: make body a func
  while curr < len(vars):
    request = base_request

    #create request
    while num_vars < 50:
      request += ','+vars[curr]
      curr += 1
      num_vars += 1
      #print(num_vars) #todo:delete
    request += limiter

    #request data
    print(request) #todo:delete
    response = requests.get(request)
    json_data = json.dumps(response.json())
    new_df = pd.read_json(json_data)
    new_df = new_df.rename(columns=new_df.iloc[0]).loc[1:].reset_index(drop=True)
    print("requested data:")
    print(new_df)

    #pearson correlation coefficient analysis w target (PINCP)

    #todo:compare FOD1P and FOD2P if they're the exact same just drop but if diff, figure out how to merge
    '''if "FOD1P" in new_df.columns and "FOD2P" in new_df.columns:
      diff = new_df["FOD1P"].compare(new_df["FOD2P"])
      print("diff")
      print(diff)''' #tbh if one correlates and one doesn't then that resolves ur issue

    #deal with problem vars

    #todo:once u go thru each var individually, convert all the if statements
    #for N/0 to be just replace across the entire df (unless u find a var that has both 0 and N)
    new_df.drop([housing_identifier, person_identifier],axis=1,inplace=True)
    if "FOD1P" in new_df.columns:
      new_df["FOD1P"] = new_df["FOD1P"].replace('N', 0)
    if "FOD2P" in new_df.columns:
      new_df["FOD2P"] = new_df["FOD2P"].replace('N', 0)
    if "NAICSP" in new_df.columns: #duplicate of indp
      new_df.drop("NAICSP", inplace = True, axis=1)
    if "INDP" in new_df.columns:
      new_df["INDP"] = new_df["INDP"].replace('N', 0)
    if "SOCP" in new_df.columns:
      new_df["SOCP"] = recode(new_df, "SOCP")
    if "LANP" in new_df.columns:
      new_df["LANP"] = new_df["LANP"].replace('N', 0)
    if "POWPUMA" in new_df.columns:
      new_df["POWPUMA"] = new_df["POWPUMA"].replace('N', 0)
    if "MIGSP" in new_df.columns:
      new_df["MIGSP"] = new_df["MIGSP"].replace('N', 0)
    if "HHLANP" in new_df.columns:
      new_df["HHLANP"] = new_df["HHLANP"].replace('N', 0)
    if "MIGPUMA" in new_df.columns:
      new_df["MIGPUMA"] = new_df["MIGPUMA"].replace('N', 0)

    #go thru vars and see what can cast (todo;delete)
    for col in new_df.columns:
      print(col)
      try:
          pd.to_numeric(new_df[col])
      except ValueError:
          print(col + " couldn't be casted")

    threshold = 0.2 #todo: change?
    corr = abs(new_df.corr(method='pearson', numeric_only=False)) #absolute val

    target_corr = corr[target].sort_values(ascending=False)[1:]
    features = target_corr[target_corr>=threshold]
    print("selected features:")
    print(features)

    #pca (todo)

    #filter new_df
    new_df = new_df[features.index]
    new_df[target] = target_col
    new_df[weight] = weight_col
    new_df[housing_identifier] = housing_identifier_col
    new_df[person_identifier] = person_identifier_col

    print("with selected features:")
    print(new_df)

    #join new_df with existing df
    df = pd.merge(df, new_df, on=[housing_identifier, person_identifier, weight, target])
    df.drop_duplicates(keep='first', inplace=True, ignore_index=True)
    print("merged")
    print(df)

    #pca on whole df (todo)

    #reset request
    request = base_request
    num_vars = 4

  df.to_csv(target + "_" + limiter + ".csv", sep=',', index=False, encoding='utf-8') #save to csv

In [46]:
def recode(df, col):
  '''
  returns recoded col
  '''
  unique = dict(enumerate(df["SOCP"].unique()))
  unique = dict([(value, key) for key, value in unique.items()])
  return df[col].replace(unique)

In [47]:
curr_vars = ["PINCP", "PWGTP", "SERIALNO", "HHLDRAGEP", "SSIP", "ELEP", "RACNUM", "WATP", "MHP", "RETP",
        "SSP", "HINCP", "RMSP", "INTP", "SEMP", "SMP", "PERNP", "PAP", "GASP",
        "WKWN", "WAGP", "FULP", "SMOCP", "FINCP", "OIP", "TAXAMT", "CONP",
        "INSP", "OCPIP", "GRNTP", "MRGP", "VALP", "BDSP", "NOC", "NP", "NRC",
        "SPORDER", "NPF", "RNTP", "WKHP", "POVPIP", "GRPIP", "JWMNP", "AGEP",
        "ADJHSG", "ADJINC", "MV", "FPARC", "DRIVESP", "RACSOR",
        "NATIVITY", "JWAP", "HICOV", "PRIVCOV", "R60", "RELSHIPP", "VACDUR",
        "MLPIK", "PLM", "VPS", "DEAR", "R18", "MLPJ", "GCL", "STOV", "TEL",
        "ELEFP", "WATFP", "YOEP", "SMX", "OTHSVCEX", "MLPCD", "ANC2P",
        "FHINS4C", "WRK", "POBP", "RACAIAN", "LAPTOP", "HHT2", "MLPFG",
        "FOD1P", "FOD2P", "SMARTPHONE", "NAICSP", "WAOB", "SOCP", "GASFP", "HIMRKS",
        "FHINS3C", "FHINS5C", "ACCESSINET", "HOTWAT", "NWLA", "CITWP",
        "JWTRNS", "REFR", "PSF", "DECADE", "PUBCOV", "FULFP", "MRGT", "INDP",
        "VACOTH", "BROADBND", "LANP", "ANC1P", "TEN", "POWPUMA",
        "HISPEED", "PLMPRP", "CPLT", "YRBLT", "DRAT", "NR", "MRGX", "HINS7",
        "MARHYP", "COMPOTHX", "SINK", "RAC2P", "RAC3P", "RAC1P", "MARHT", "SATELLITE", "WIF",
        "HISP", "MAR", "SCHL", "NWLK", "DPHY", "DEYE", "MIGSP",
        "HHLANP", "PARTNER", "RACNH", "WKL", "VEH", "DDRS", "MIGPUMA", "LNGI",
        "HINS2", "QTRBIR", "SFN", "RACBLK", "MLPH", "ESR", "NPP", "DIS",
        "DIALUP", "HHLDRRAC1P", "TABLET", "MLPB", "DOUT", "SCH",
        "RACPI", "POWSP", "ANC", "MIL", "OC", "HUGCL", "RWAT", "HHLDRHISP",
        "HINS3", "RESMODE", "MARHW", "SFR", "ESP", "RACASN", "HINS5", "MLPE",
        "OCCP", "MARHD", "SCHG", "MRGI", "MIG", "HINS1", "MSP", "FER",
        "MULTG", "WORKSTAT", "MARHM", "KIT", "GCR", "HUPARC", "HINS6",
        "GCM", "ACR", "HINS4", "PAOC", "RNTM", "DRATX", "FS", "SVAL",
        "RACWHT", "NWAB", "HUPAOC", "R65", "RC", "BATH", "SEX", "HFL",
        "WKEXREL", "VACS", "HHL", "SRNT", "NWAV", "NWRE", "BLD", "LANX",
        "MLPA", "HHT", "DREM", "COW", "HUPAC", "CIT", "AGS", "ENG", "JWRIP",
        "JWDP", "NOP"]

In [48]:
#aapi
import_census_data(curr_vars, "PINCP", "2021", "&RACASN=1&RACPI=1&ucgid=0400000US06") #todo: try limiting not just to CA

https://api.census.gov/data/2021/acs/acs1/pums?get=PWGTP,PINCP,SERIALNO,SPORDER&RACASN=1&RACPI=1&ucgid=0400000US06
      PINCP PWGTP       SERIALNO SPORDER
0      5000    84  2021GQ0012765       1
1      3700    51  2021GQ0013564       1
2       500    77  2021GQ0027187       1
3      3700    69  2021GQ0027554       1
4         0    50  2021GQ0030297       1
..      ...   ...            ...     ...
553   20000    77  2021HU0901344       1
554  180000    92  2021HU0901344       2
555   66000    73  2021HU0901344       3
556   10500   148  2021HU0918725       1
557   10100   150  2021HU0918725       2

[558 rows x 4 columns]
https://api.census.gov/data/2021/acs/acs1/pums?get=PWGTP,PINCP,SERIALNO,SPORDER,HHLDRAGEP,SSIP,ELEP,RACNUM,WATP,MHP,RETP,SSP,HINCP,RMSP,INTP,SEMP,SMP,PERNP,PAP,GASP,WKWN,WAGP,FULP,SMOCP,FINCP,OIP,TAXAMT,CONP,INSP,OCPIP,GRNTP,MRGP,VALP,BDSP,NOC,NP,NRC,NPF,RNTP,WKHP,POVPIP,GRPIP,JWMNP,AGEP,ADJHSG,ADJINC,MV,FPARC,DRIVESP,RACSOR&RACASN=1&RACPI=1&ucgid=0400000US06
request

TypeError: ignored

In [ ]:
#predict race
import_census_data(curr_vars, "RAC1P", "2021", "&ucgid=0400000US06")

In [ ]:
#income across race
'''years = ["2005", "2007", "2009", "2011", "2013", "2015", "2017", "2019", "2021"]
#todo:u need to recode NAICSP to INDP (i think hg was helping with that) for this

for y in years:
  import_census_data(curr_vars, "PINCP", y, )''' #need to insert geographies, also probably need a diff vars list bc not all vars every year